## 라이브러리 불러오기

In [1]:
import random
import pandas as pd
import numpy as np
import os

# 사이킷런에서 랜덤포레스트 모델 불러오기
from sklearn.ensemble import RandomForestRegressor

# 경고 메세지 무시하기
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록(그래프가 화면에 뜨지 않을 때)
%matplotlib inline

# 그래프(figure)에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정(나눔고딕체)
plt.rc('font', family='NanumGothic')

# 조용하 강사님이 올려주신 코드(한글화-matplotlib)
!pip install koreanize_matplotlib
import koreanize_matplotlib

Defaulting to user installation because normal site-packages is not writeable


## 데이터 불러오기

In [3]:
train = pd.read_csv('train.csv',encoding='cp949')
test = pd.read_csv('test.csv',encoding='cp949')
submission=pd.read_csv('sample_submission.csv',encoding='cp949')

In [4]:
# 컬럼명 변환을 위한 리스트 할당
train_col = ['num', 'date_time', 'power', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
test_col = ['num', 'date_time', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
train.columns=train_col
test.columns=test_col

## 전처리

### 날짜 전처리

- 파생변수, 데이터 타입 변경

In [5]:
def make_time(train):
    """
    시간 관련 변수를 추가하기 위한 함수
    """
    train['date_time'] = pd.to_datetime(train.date_time)
    
    train['month'] = train.date_time.dt.month                    # 월(숫자)
    train['day'] = train.date_time.dt.day                        # 일(숫자)
    train['hour'] = train.date_time.dt.hour                      # 시(숫자)
    train['weekday'] = train.date_time.dt.weekday                # 요일(숫자)
    train['dayofyear'] = train.date_time.dt.dayofyear            # 연 기준 몇일째(숫자)
    #train['week']=train.date_time.dt.weekofyear
    
    ## cyclical encoding
    train['sin_time'] = np.sin(2*np.pi*train.hour/24)
    train['cos_time'] = np.cos(2*np.pi*train.hour/24)
    
    ### 공휴일 변수 추가
    train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
    train.loc[('2020-08-17'<=train.date_time)&(train.date_time<'2020-08-18'), 'holiday'] = 1
    
    
    ### 날씨 변수 추가
    train['THI'] = 1.9*train['temp'] - 0.55*(1-train['hum'])*(1.8*train['temp']-26)+32                                # 불쾌지수
    train['chill']=13.12+0.6215*train['temp']-11.37*train['wind']**0.16+0.3965*train['wind']**0.16*train['temp']        # 체감온도
    
    train['냉방도시'] = 0  # 냉방이 필요하지 않은 도시로 초기화

    # 여름 동안 냉방이 필요한 도시로 설정
    train.loc[(train['temp'] > 30) & (train['hum'] > 70), '냉방도시'] = 1
    
    return train

### 날씨 전처리

- 파생변수, 결측치

In [6]:
def make_nan(test):
    """
    날씨 관련 결측치를 처리하기 위한 함수
    """
    
    # 결측치 처리
    test['wind']=test['wind'].interpolate(method='linear')
    test['rain']=test['rain'].interpolate(method='linear')
    test['sun']=test['sun'].interpolate(method='linear')
    
    test['temp']=test['temp'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    test['hum']=test['hum'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    
    test['cooler']=test['cooler'].fillna(0)
    test['solar']=test['solar'].fillna(0)
    
    return test

In [7]:
def make_weather(data):
    weather=data.groupby(['num','dayofyear'])['temp'].agg({'max','min','mean'}).reset_index()
    weather['temp_range']=weather['max']-weather['min']
    
    data=pd.merge(data,weather,on=['num','dayofyear'],how='left')
    return data
    

In [8]:
train = make_time(train)
test=make_nan(test)
test = make_time(test)
train = make_weather(train)
test=make_weather(test)

### 전력사용량 전처리

- 파생변수

In [14]:
from tqdm import tqdm

power_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
power_hour_mean = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
power_hour_std = pd.pivot_table(train, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()

def make_power(data):
    
    # 건물별, 요일별, 시간별 발전량 평균
    #power_mean = pd.pivot_table(data, values = 'power', index = ['num', 'hour', 'day'], aggfunc = np.mean).reset_index()
    tqdm.pandas()
    data['day_hour_mean'] = data.progress_apply(lambda x : power_mean.loc[(power_mean.num == x['num']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'power'].values[0], axis = 1)
    
    # 건물별 시간별 발전량 평균 넣어주기
    #power_hour_mean = pd.pivot_table(data, values = 'power', index = ['num', 'hour'], aggfunc = np.mean).reset_index()
    tqdm.pandas()
    data['hour_mean'] = data.progress_apply(lambda x : power_hour_mean.loc[(power_hour_mean.num == x['num']) & (power_hour_mean.hour == x['hour']) ,'power'].values[0], axis = 1)
    
    # 건물별 시간별 발전량 표준편차 넣어주기
    #power_hour_std = pd.pivot_table(data, values = 'power', index = ['num', 'hour'], aggfunc = np.std).reset_index()
    data['hour_std'] = data.progress_apply(lambda x : power_hour_std.loc[(power_hour_std.num == x['num']) & (power_hour_std.hour == x['hour']) ,'power'].values[0], axis = 1)
    return data

In [15]:
train = make_power(train)

100%|██████████| 122400/122400 [00:36<00:00, 3340.31it/s]


In [16]:
test=make_power(test)

100%|██████████| 10080/10080 [00:02<00:00, 3362.69it/s]


# 시계열 관련 모델

## Prophet모델

In [ ]:
import random
import pandas as pd
import numpy as np
import os

# 사이킷런에서 랜덤포레스트 모델 불러오기
from sklearn.ensemble import RandomForestRegressor

# 경고 메세지 무시하기
import warnings
warnings.filterwarnings(action='ignore')

!pip install koreanize_matplotlib

In [ ]:
import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록(그래프가 화면에 뜨지 않을 때)
%matplotlib inline

# 그래프(figure)에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정(나눔고딕체)
plt.rc('font', family='NanumGothic')

# 조용하 강사님이 올려주신 코드(한글화-matplotlib)
import koreanize_matplotlib

In [ ]:
if 'google.colab' in str(get_ipython()):
    ! pip install git+https://github.com/ourownstory/neural_prophet.git # 코랩 구동 시 패키지 설치
    # pip install neuralprophet
    # 간단하고 빠르게 설치할 수 있지만, 최신 버전에서의 버그가 발생할 수 있음

In [ ]:
#패키지 임포트
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
set_log_level("ERROR")

#구글 드라이브 다운로드 패키지
from google_drive_downloader import GoogleDriveDownloader as gdd

#데이터 다운로드
gdd.download_file_from_google_drive(file_id='1JRslcolP0XJDQKfXrYl82eyDTCETuzjl',
                                    dest_path='data/energy.zip',
                                    unzip=True)

In [ ]:
train = pd.read_csv('train.csv',encoding='cp949')
test = pd.read_csv('test.csv',encoding='cp949')

In [ ]:
# 컬럼명 변환을 위한 리스트 할당
train_col = ['num', 'date_time', 'power', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
test_col = ['num', 'date_time', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
train.columns=train_col
test.columns=test_col

In [ ]:
def make_nan(test):

    # 결측치 처리
    test['wind']=test['wind'].interpolate(method='linear')
    test['rain']=test['rain'].interpolate(method='linear')
    test['sun']=test['sun'].interpolate(method='linear')

    test['temp']=test['temp'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    test['hum']=test['hum'].interpolate(method='polynomial',order=3).fillna(method='ffill')

    test['cooler']=test['cooler'].fillna(0)
    test['solar']=test['solar'].fillna(0)

    return test

In [ ]:
test=make_nan(test)

In [ ]:
pip install neuralprophet

In [ ]:
pred=[]
for i in range(1,61):
  train_d = pd.DataFrame({"ds": train[train['num']==i]["date_time"], "y": train[train['num']==i]["power"]})
  tes_dt = pd.DataFrame({"ds": test[test['num']==i]["date_time"], "y": test[test['num']==i]["power"]})
  #모델 설정
  m = NeuralProphet(
      learning_rate=0.1,
  )
  #모델 학습
  metrics = m.fit(train_d, freq="H")
  test_pred = m.predict(tes_dt)
  pred.append(test_pred)



In [ ]:
# 'yhat1' 열을 추출할 리스트
yhat1_list = []

# pred 리스트에 있는 각 데이터프레임에서 'yhat1' 열을 추출
for df in pred:
    yhat1_list.append(df['yhat1'])

In [ ]:
submission=pd.read_csv('sample_submission.csv',encoding='cp949')

In [ ]:
submission['answer']=pd.DataFrame(np.array(yhat1_list).flatten())

## LSTM

In [5]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

#######딥러닝 라이브러리##########
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64')

2023-10-31 06:48:59.655606: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-31 06:48:59.655671: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-31 06:48:59.655697: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-31 06:48:59.663518: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 06:49:00.482915: W tensorflow/compiler/

In [6]:
# 맵플롯립 시각화 및 한국어 설정 (맷플롯립 시각화 시)
import matplotlib.pyplot as plt

# 브라우저에서 바로 그려지도록(그래프가 화면에 뜨지 않을 때)
%matplotlib inline

# 그래프(figure)에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정(나눔고딕체)
plt.rc('font', family='NanumGothic')

# 조용하 강사님이 올려주신 코드(한글화-matplotlib)
!pip install koreanize-matplotlib
import koreanize_matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [7]:
train=pd.read_csv('train.csv', encoding = 'cp949')
test=pd.read_csv('test.csv', encoding = 'cp949')
submission=pd.read_csv('sample_submission.csv', encoding = 'cp949')

In [8]:
# 컬럼명 변환을 위한 리스트 할당
train_col = ['num', 'date_time', 'power', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
test_col = ['num', 'date_time', 'temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'solar']
train.columns=train_col
test.columns=test_col

In [9]:
def make_time(train):
    """
    시간 관련 변수를 추가하기 위한 함수
    """
    train['date_time'] = pd.to_datetime(train.date_time)
    
    train['month'] = train.date_time.dt.month                    # 월(숫자)
    train['day'] = train.date_time.dt.day                        # 일(숫자)
    train['hour'] = train.date_time.dt.hour                      # 시(숫자)
    train['weekday'] = train.date_time.dt.weekday                # 요일(숫자)
    train['dayofyear'] = train.date_time.dt.dayofyear            # 연 기준 몇일째(숫자)
    # train['week']=train.date_time.dt.weekofyear
    
    ## cyclical encoding
    train['sin_time'] = np.sin(2*np.pi*train.hour/24)
    train['cos_time'] = np.cos(2*np.pi*train.hour/24)
    
    ### 공휴일 변수 추가
    train['holiday'] = train.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
    train.loc[('2020-08-17'<=train.date_time)&(train.date_time<'2020-08-18'), 'holiday'] = 1
    
    
    ### 날씨 변수 추가
    train['THI'] = 1.9*train['temp'] - 0.55*(1-train['hum'])*(1.8*train['temp']-26)+32                                # 불쾌지수
    train['chill']=13.12+0.6215*train['temp']-11.37*train['wind']**0.16+0.3965*train['wind']**0.16*train['temp']        # 체감온도
    
    train['냉방도시'] = 0  # 냉방이 필요하지 않은 도시로 초기화

    # 여름 동안 냉방이 필요한 도시로 설정
    train.loc[(train['temp'] > 30) & (train['hum'] > 70), '냉방도시'] = 1
    
    return train

In [10]:
def make_nan(test):
    
    # 결측치 처리
    test['wind']=test['wind'].interpolate(method='linear')
    test['rain']=test['rain'].interpolate(method='linear')
    test['sun']=test['sun'].interpolate(method='linear')
    
    test['temp']=test['temp'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    test['hum']=test['hum'].interpolate(method='polynomial',order=3).fillna(method='ffill')
    
    test['cooler']=test['cooler'].fillna(0)
    test['solar']=test['solar'].fillna(0)
    
    return test

In [11]:
train = make_time(train)
#train = make_power(train)
test = make_nan(test)
test = make_time(test)

In [12]:
def make_weather(data):
    weather=data.groupby(['num','dayofyear'])['temp'].agg({'max','min','mean'}).reset_index()
    weather['temp_range']=weather['max']-weather['min']
    
    data=pd.merge(data,weather,on=['num','dayofyear'],how='left')
    return data

In [13]:
#train = make_power(train)
train = make_weather(train)
test=make_weather(test)

In [14]:
train=train[train['num']==1]
test=test[test['num']==1]

In [15]:
from sklearn.preprocessing import MinMaxScaler

# 입력 변수 선택
feature_cols = ['temp', 'wind', 'hum', 'rain', 'sun', 'cooler', 'holiday', 'THI', 'chill', '냉방도시', 'mean', 'min', 'max', 'temp_range','date_time']
## 종속변수 선택
label_cols = ['power']

In [16]:
feature_df = pd.DataFrame(train[feature_cols], columns=feature_cols).set_index('date_time')
label_df = pd.DataFrame(train[label_cols], columns=label_cols)

In [21]:
## 딥러닝 학습을 위해 numpy array로 변환
feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

In [24]:
feature_np.shape

(2040, 14)

In [19]:
label_df

,power
0,8179.056
1,8135.640
2,8107.128
3,8048.808
4,8043.624
...,...
2035,8714.952
2036,8740.224
2037,8730.504
2038,8725.968


In [23]:
np.random.rand(1000, 10, 1).shape

(1000, 10, 1)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed

# 가상의 시계열 데이터 생성
def generate_time_series(n_samples, n_timesteps, n_features, n_outputs):
    X = np.random.rand(n_samples, n_timesteps, n_features)
    y = np.random.rand(n_samples, n_outputs, n_features)
    return X, y

n_samples = label_df.shape[0]
n_timesteps = 168
n_features = 10
n_outputs = 168  # 예제에서는 다음 5개 스텝을 예측

X, y = generate_time_series(n_samples, n_timesteps, n_features, n_outputs)

# 모델 구축
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_timesteps, n_features)))
model.add(TimeDistributed(Dense(n_features))  # TimeDistributed 레이어를 사용하여 각 스텝에 대한 출력을 생성
model.compile(optimizer='adam', loss='mse')  # 회귀 문제이므로 평균 제곱 오차를 사용합니다.

# 모델 훈련
model.fit(X, y, epochs=10, batch_size=32)

# 새로운 데이터에 대한 예측
new_data = np.random.rand(1, n_timesteps, n_features)
prediction = model.predict(new_data)

print(f'Prediction: {prediction}')


## Arima

In [ ]:
#Library Imports
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error


#######통계 라이브러리##########
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
#2d의 데이터프레임을 건물별 정보를 반영한 3d 데이터로 변환
def df2d_to_array3d(df_2d):
    feature_size=df_2d.iloc[:,2:].shape[1]
    time_size=len(df_2d['date_time'].value_counts())
    sample_size=len(df_2d.num.value_counts())
    return df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])

In [ ]:
train_x_array=df2d_to_array3d(train)
test_x_array=df2d_to_array3d(test)

In [ ]:
def plot_series(x_series, y_series):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'output_series')
    plt.axhline(1, c = 'red')
    plt.legend()

In [ ]:
idx=1
x_series=train_x_array[idx, :, 0]
model=ARIMA(x_series, order=(3, 0, 1))
fit=model.fit()

In [ ]:
preds=fit.predict(1, 168, typ='levels')

In [ ]:
plt.plot(x_series, label = 'input_series')
plt.plot(np.arange(2040, 2040+168), test_x_array[idx, :, 0], label='y')
plt.plot(np.arange(2040, 2040+168), preds, label='prediction')
plt.legend()

In [ ]:
valid_pred_array=np.zeros([60, 168])
for idx in range(train_x_array.shape[0]):
    try:
        try:
            x_series=train_x_array[idx, :, 0]
            model=ARIMA(x_series, order=(5, 1, 1))
            fit=model.fit()
            preds=fit.predict(1, 168, typ='levels')
            valid_pred_array[idx, :]=preds
        except:
            print("order 4,1,1")
            
            x_series=train_x_array[idx, :, 0]
            model=ARIMA(x_series, order=(4, 1, 1))
            fit=model.fit()
            preds=fit.predict(1, 168, typ='levels')
            valid_pred_array[idx, :]=preds
    except:
        print(idx, "샘플은 수렴하지 않습니다.")

In [ ]:
submission['answer']=valid_pred_array.reshape([-1,1])
submission

In [ ]:
submission.to_csv('baseline_submission2.csv', index=False)